In [1]:
#default_exp utils.viewer.pcviewer

In [2]:
#export
import torch
import math
import numpy as np
import open3d as o3d

#from pointpillars.utils.box_ops import convert_boxes_to_3d_corners

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
#export
class PCViewer(object):

    def __init__(self):
        pass

    @staticmethod
    def view_pc_with_jupyter():
        """
        Visualize the point cloud on Jupyter Notebook.
        """
        pass

    @staticmethod
    def line_set_from_3d_box(box: np.ndarray, color: list) -> o3d.geometry.LineSet:
        """

        :param box: Tensor(1,7) with x,y,z,h,w,l,theta
        :param colors: list [3] with the rgb values between 0 and 1
        :return: LineSet as bounding box
        """
        x, y, z, h, w, l, theta = box.tolist()

        #dh = 0.5 * h
        dw = 0.5 * w
        dl = 0.5 * l

        p0 = [x - dl, y + dw, z]
        p1 = [x - dl, y - dw, z]
        p2 = [x + dl, y - dw, z]
        p3 = [x + dl, y + dw, z]
        p4 = [x - dl, y + dw, z + h]
        p5 = [x - dl, y - dw, z + h]
        p6 = [x + dl, y - dw, z + h]
        p7 = [x + dl, y + dw, z + h]

        points = [p0, p1, p2, p3, p4, p5, p6, p7]
        #points = PCViewer.rotate_points_around_z(points, theta)
        # indices which corners have a connection with each other
        lines = [[0, 1], [0, 3], [2, 1], [2, 3], [4, 7], [4, 5], [6, 5], [6, 7], [0, 4], [1, 5], [2, 6], [3, 7]]

        colors = [color for i in range(len(lines))]

        line_set = o3d.geometry.LineSet()
        line_set.points = o3d.utility.Vector3dVector(points)
        line_set.lines =  o3d.utility.Vector2iVector(lines)
        line_set.colors = o3d.utility.Vector3dVector(colors)


        return line_set

    @staticmethod
    def rotate_points_around_z(points: list, theta: float):
        """
        Rotates corner points by an angle theta around the z axes.
        The formulas are: Rv = (xcos(theta) - ysin(theta))e_x + (xsin(theta) + ycos(theta))e_y

        :param points: a list containing all points in format [[x1,y1,z1], [x2, y2, z2], ...]
        :param theta: angle by which to rotate. Given in rad.

        :returns: list containing the newly rotated points in format [[xn1, yn1, z1], [xn2, yn2, z2], ...]
        """
        for i, point in enumerate(points):
            x, y, z = point

            x_new = x * math.cos(theta) - y * math.sin(theta)
            y_new = x * math.sin(theta) + y * math.cos(theta)

            point = [x_new, y_new, z]
            points[i] = point

        return points

    @staticmethod
    def view_pcd_from_network_output(pcloud: torch.Tensor, pred_boxes: torch.Tensor, gt_boxes: torch.Tensor):
        """
        #TODO: Confirm dimension order from points
        :param pcloud: Tensor(nb_points, 4) with last dim being x,y,z,r
        :param pred_boxes: Tensor(nb_pred_boxes, 7) with last dim being x,y,z,h,w,l,theta
        :param gt_boxes: Tensor(nb_gt_boxes, 7) with last dim being x,y,z,h,w,l,theta
        :return:
        """
        pcloud = pcloud.cpu()
        pred_boxes = pred_boxes.cpu()
        gt_boxes = gt_boxes.cpu()

        pcloud = pcloud.numpy()
        pred_boxes = pred_boxes.numpy()
        gt_boxes = gt_boxes.numpy()

        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.open3d.utility.Vector3dVector(pcloud[:,:3])

        pred_boxes_arr = []
        for i, pred_box in enumerate(pred_boxes):
            pred_line_set = PCViewer.line_set_from_3d_box(pred_box, [0,0,1])
            pred_boxes_arr.append(pred_line_set)

        gt_boxes_arr = []
        for i, gt_box in enumerate(gt_boxes):
            gt_line_set = PCViewer.line_set_from_3d_box(gt_box, [0,1,0])
            gt_boxes_arr.append(gt_line_set)

        geometries = pred_boxes_arr + gt_boxes_arr
        #geometries = gt_boxes_arr
        geometries.append(pcd)
        o3d.visualization.draw_geometries(geometries)



In [4]:
#export
img_nbr = '000100'
velodyne_location = '/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/velodyne/training/{}.bin'.format(img_nbr)


In [5]:
#hide
pred_boxes = torch.Tensor([[0,0,0,2,2,2,2], [5,5,5,5,5,5,5], [0,0,0,1,2,1,0]])

pcviewer = PCViewer()
pcviewer.pc_from_network_output(pred_boxes)


AttributeError: 'PCViewer' object has no attribute 'pc_from_network_output'

In [6]:
#hide
points = [[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0], [0, 0, 1], [1, 0, 1],
              [0, 1, 1], [1, 1, 1]]
lines = [[0, 1], [0, 2], [1, 3], [2, 3], [4, 5], [4, 6], [5, 7], [6, 7],
             [0, 4], [1, 5], [2, 6], [3, 7]]
colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(points)
line_set.lines = o3d.utility.Vector2iVector(lines)
line_set.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([line_set])

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [7]:
#hide
# Load binary point cloud
bin_pcd = np.fromfile(velodyne_location, dtype=np.float32)

# Reshape and drop reflection values
points = bin_pcd.reshape((-1, 4))[:, 0:3]

# Convert to Open3D point cloud
pcd_pseudo = o3d.open3d.geometry.PointCloud()
pcd_pseudo.points = o3d.open3d.utility.Vector3dVector(points)

o3d.visualization.draw_geometries([pcd_pseudo])



[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
